In [1]:
#Import libraries
from pyhive import presto
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import h3
from shapely.geometry import Polygon, Point
import calendar
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 500)

In [2]:
#Define presto credentials
presto_host = 'bi-presto.serving.data.production.internal'
presto_port = '80'
username = 'aditya.bhattar@rapido.bike'

#Create connection to presto host
connection = presto.connect(presto_host,presto_port,username = username)

In [3]:
#Query data from order_logs_snapshot

q = """
select order_id, accept_to_pickup_distance, captain_location_hex_8, captain_location_latitude,
captain_location_longitude, city_name, customer_location_hex_8, customer_location_latitude, 
pickup_location_hex_8, pickup_location_latitude, pickup_location_longitude, drop_location_hex_8, drop_location_latitude, drop_location_longitude,
customer_location_longitude, hhmmss, hour, quarter_hour, service_obj_service_name, time_bucket, weekday, yyyymmdd,
order_status, cancel_reason
from orders.order_logs_snapshot
where yyyymmdd >= '20230306'
and yyyymmdd <= '20230310'
and city_name in ('Hyderabad')
and service_obj_service_name = 'Link'
and hour in ('08', '09', '10', '11', '17', '18', '19', '20', '21')
"""

#Load data into pandas table
df_copy = pd.read_sql(q, connection)

In [17]:
#Create a copy of the df retrived from presto
df = df_copy.copy()
df.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,customer_location_latitude,pickup_location_hex_8,pickup_location_latitude,pickup_location_longitude,drop_location_hex_8,drop_location_latitude,drop_location_longitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason
0,6405e723dffc1036eb142200,1.765,8860b52e89fffff,17.376173,78.553596,Hyderabad,8860a25b61fffff,17.396656,8860a25b61fffff,17.396582,78.514220,8860b52e89fffff,17.376173,78.553596,78.514290,184411,18,1830,Link,18:59,1,20230306,dropped,None
1,6405e72cce0f880cb87f85d8,0.338,8860a259e5fffff,17.454533,78.424805,Hyderabad,8860b196ebfffff,17.491140,8860b196ebfffff,17.491148,78.450874,8860a259e5fffff,17.454533,78.424805,78.450874,184420,18,1830,Link,18:59,1,20230306,dropped,Long Pickup
2,6405e7581215c13df1c82fe1,0.621,8860a24921fffff,17.558580,78.347733,Hyderabad,8860a24921fffff,17.558582,8860b196d9fffff,17.456558,78.433543,8860b196d9fffff,17.457048,78.433361,78.347740,184504,18,1845,Link,18:59,1,20230306,customerCancelled,Found better price elsewhere
3,6405e78f01b8b86aa8bd578d,0.374,8860a25963fffff,17.446512,78.476143,Hyderabad,8860a25903fffff,17.434916,8860a25903fffff,17.434938,78.450502,8860a25963fffff,17.446512,78.476143,78.450478,184559,18,1845,Link,18:59,1,20230306,dropped,None
4,6405e7b1906657075b0e1ae3,2.733,8860b196e5fffff,17.499882,78.459274,Hyderabad,8860b196cdfffff,17.469910,8860b196cdfffff,17.469909,78.458405,8860b196e5fffff,17.499882,78.459274,78.458405,184633,18,1845,Link,18:59,1,20230306,dropped,None


In [18]:
#Make columns for second and minute and date
df['second'] = df['hhmmss'].apply(lambda x: x[4:])
df['minute'] = df['hhmmss'].apply(lambda x: x[2:4])
df['date'] = df['yyyymmdd'].apply(lambda x: x[6:])
df['count'] = 1
df.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,customer_location_latitude,pickup_location_hex_8,pickup_location_latitude,pickup_location_longitude,drop_location_hex_8,drop_location_latitude,drop_location_longitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,second,minute,date,count
0,6405e723dffc1036eb142200,1.765,8860b52e89fffff,17.376173,78.553596,Hyderabad,8860a25b61fffff,17.396656,8860a25b61fffff,17.396582,78.514220,8860b52e89fffff,17.376173,78.553596,78.514290,184411,18,1830,Link,18:59,1,20230306,dropped,None,11,44,06,1
1,6405e72cce0f880cb87f85d8,0.338,8860a259e5fffff,17.454533,78.424805,Hyderabad,8860b196ebfffff,17.491140,8860b196ebfffff,17.491148,78.450874,8860a259e5fffff,17.454533,78.424805,78.450874,184420,18,1830,Link,18:59,1,20230306,dropped,Long Pickup,20,44,06,1
2,6405e7581215c13df1c82fe1,0.621,8860a24921fffff,17.558580,78.347733,Hyderabad,8860a24921fffff,17.558582,8860b196d9fffff,17.456558,78.433543,8860b196d9fffff,17.457048,78.433361,78.347740,184504,18,1845,Link,18:59,1,20230306,customerCancelled,Found better price elsewhere,04,45,06,1
3,6405e78f01b8b86aa8bd578d,0.374,8860a25963fffff,17.446512,78.476143,Hyderabad,8860a25903fffff,17.434916,8860a25903fffff,17.434938,78.450502,8860a25963fffff,17.446512,78.476143,78.450478,184559,18,1845,Link,18:59,1,20230306,dropped,None,59,45,06,1
4,6405e7b1906657075b0e1ae3,2.733,8860b196e5fffff,17.499882,78.459274,Hyderabad,8860b196cdfffff,17.469910,8860b196cdfffff,17.469909,78.458405,8860b196e5fffff,17.499882,78.459274,78.458405,184633,18,1845,Link,18:59,1,20230306,dropped,None,33,46,06,1


In [6]:
#Load data from cluster table
q = """
select * from datasets.city_cluster_hex
where city in ('Hyderabad')
and resolution = 8
"""
#Load data into pandas table
df_cluster_copy = pd.read_sql(q, connection)

In [19]:
cluster_mapping = df_cluster_copy.copy()
cluster_mapping = cluster_mapping.rename(columns = {'hex_id':'pickup_location_hex_8'})
df = df.merge(cluster_mapping[['pickup_location_hex_8', 'cluster']], on = 'pickup_location_hex_8', how = 'left')
cluster_mapping = cluster_mapping.rename(columns = {'pickup_location_hex_8':'drop_location_hex_8'})
df = df.rename(columns = {'cluster':'pickup_cluster'})
df = df.merge(cluster_mapping[['drop_location_hex_8', 'cluster']], on = 'drop_location_hex_8', how = 'left')
df = df.rename(columns = {'cluster':'drop_cluster'})
df.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,customer_location_latitude,pickup_location_hex_8,pickup_location_latitude,pickup_location_longitude,drop_location_hex_8,drop_location_latitude,drop_location_longitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,second,minute,date,count,pickup_cluster,drop_cluster
0,6405e723dffc1036eb142200,1.765,8860b52e89fffff,17.376173,78.553596,Hyderabad,8860a25b61fffff,17.396656,8860a25b61fffff,17.396582,78.514220,8860b52e89fffff,17.376173,78.553596,78.514290,184411,18,1830,Link,18:59,1,20230306,dropped,None,11,44,06,1,Amberpet,Nagole
1,6405e72cce0f880cb87f85d8,0.338,8860a259e5fffff,17.454533,78.424805,Hyderabad,8860b196ebfffff,17.491140,8860b196ebfffff,17.491148,78.450874,8860a259e5fffff,17.454533,78.424805,78.450874,184420,18,1830,Link,18:59,1,20230306,dropped,Long Pickup,20,44,06,1,Quthbullapur,Mothinagar
2,6405e7581215c13df1c82fe1,0.621,8860a24921fffff,17.558580,78.347733,Hyderabad,8860a24921fffff,17.558582,8860b196d9fffff,17.456558,78.433543,8860b196d9fffff,17.457048,78.433361,78.347740,184504,18,1845,Link,18:59,1,20230306,customerCancelled,Found better price elsewhere,04,45,06,1,Erragadda Sunday Market,Erragadda Sunday Market
3,6405e78f01b8b86aa8bd578d,0.374,8860a25963fffff,17.446512,78.476143,Hyderabad,8860a25903fffff,17.434916,8860a25903fffff,17.434938,78.450502,8860a25963fffff,17.446512,78.476143,78.450478,184559,18,1845,Link,18:59,1,20230306,dropped,None,59,45,06,1,Begumpet,Begumpet
4,6405e7b1906657075b0e1ae3,2.733,8860b196e5fffff,17.499882,78.459274,Hyderabad,8860b196cdfffff,17.469910,8860b196cdfffff,17.469909,78.458405,8860b196e5fffff,17.499882,78.459274,78.458405,184633,18,1845,Link,18:59,1,20230306,dropped,None,33,46,06,1,Quthbullapur,Jeedimetla Industrial Estate


In [20]:
#Filter the df on morning and evening peak hours
df_morning = df[df['hour'].isin(['08', '09', '10', '11'])]
df_evening = df[df['hour'].isin(['17', '18', '19', '20', '21'])]

#Groupby on hex and quarter hour to get the number of orders coming in every hex
df_morning_grouped = df_morning.groupby(['yyyymmdd', 'hour', 'pickup_location_hex_8'])['order_id'].nunique().reset_index()
df_evening_grouped = df_evening.groupby(['yyyymmdd', 'hour', 'pickup_location_hex_8'])['order_id'].nunique().reset_index()
df_morning_grouped = df_morning_grouped.rename(columns = {'order_id':'num_of_orders'})
df_evening_grouped = df_evening_grouped.rename(columns = {'order_id':'num_of_orders'})
df_evening_grouped.head()

,yyyymmdd,hour,pickup_location_hex_8,num_of_orders
0,20230306,17,8860a2410dfffff,2
1,20230306,17,8860a2412dfffff,4
2,20230306,17,8860a2413dfffff,1
3,20230306,17,8860a24141fffff,3
4,20230306,17,8860a24153fffff,1


In [21]:
df_morning = df_morning.merge(df_morning_grouped, on = ['yyyymmdd', 'hour', 'pickup_location_hex_8'], how = 'left')
df_evening = df_evening.merge(df_evening_grouped, on = ['yyyymmdd', 'hour', 'pickup_location_hex_8'], how = 'left')

In [22]:
df_morning['weekday_name'] = df_morning['weekday'].apply(lambda x: calendar.day_name[(x-1)])
df_evening['weekday_name'] = df_evening['weekday'].apply(lambda x: calendar.day_name[(x-1)])

In [11]:
#Query data for mismatch

q_online_captains = """
select * from pricing.cluster_temporal_hotness
where service_detail_id = '574013f14fdf4798208bba26'
"""


#Load data into pandas table
df_mismatch_copy = pd.read_sql(q_online_captains, connection)

In [23]:
#View the dataset
df_mismatch = df_mismatch_copy.copy()
df_mismatch.head()

,hotness_category,pickupclusters,week_period,time_period,temporal_id,total_footfall,total_revenue,hex_count,total_revenue_normalized,normalized_revenue_contribution_percentage,cum_revenue_contribution_percentage,revenue_contribution_percentile_rank,temporal_stress,temporal_mismatch,spatial_stress,spatial_mismatch,cluster_hotness_score,run_date,strategy_version,service_detail_id
0,cold,Alwal,Tuesday,2200,Tuesday-2200,0,0.0,0,0.0,0.0,0.0,0.0,0.394594,0.240619,0.163608,0.099372,0.224548,20230228,HOT_COLD_3_0,574013f14fdf4798208bba26
1,medium,Alwal,Tuesday,2000,Tuesday-2000,0,0.0,0,0.0,0.0,0.0,0.0,0.558497,0.339180,0.251984,0.172206,0.330467,20230228,HOT_COLD_3_0,574013f14fdf4798208bba26
2,medium,Alwal,Tuesday,0700,Tuesday-0700,0,0.0,0,0.0,0.0,0.0,0.0,0.681962,0.690130,0.568700,0.524388,0.616295,20230228,HOT_COLD_3_0,574013f14fdf4798208bba26
3,medium,Alwal,Tuesday,1900,Tuesday-1900,0,0.0,0,0.0,0.0,0.0,0.0,0.543719,0.394297,0.292824,0.219659,0.362625,20230228,HOT_COLD_3_0,574013f14fdf4798208bba26
4,cold,Alwal,Tuesday,2300,Tuesday-2300,0,0.0,0,0.0,0.0,0.0,0.0,0.463224,0.286727,0.154597,0.109623,0.253543,20230228,HOT_COLD_3_0,574013f14fdf4798208bba26


In [24]:
df_mismatch = df_mismatch[df_mismatch['run_date'] == df_mismatch['run_date'].max()]

In [25]:
#rename column
df_mismatch = df_mismatch.rename(columns = {'week_period':'weekday_name', 'pickupclusters':'pickup_cluster'})
df_mismatch['hour'] = df_mismatch['time_period'].apply(lambda x: x[:2])
df_mismatch.head()

,hotness_category,pickup_cluster,weekday_name,time_period,temporal_id,total_footfall,total_revenue,hex_count,total_revenue_normalized,normalized_revenue_contribution_percentage,cum_revenue_contribution_percentage,revenue_contribution_percentile_rank,temporal_stress,temporal_mismatch,spatial_stress,spatial_mismatch,cluster_hotness_score,run_date,strategy_version,service_detail_id,hour
141017,medium,Ambar Chreuvu,Thursday,2100,Thursday-2100,0,0.0,0,0.0,0.0,0.0,0.0,0.400945,0.520092,0.273786,0.228479,0.355826,20230314,HOT_COLD_3_0,574013f14fdf4798208bba26,21
142149,cold,Ambar Chreuvu,Thursday,2300,Thursday-2300,0,0.0,0,0.0,0.0,0.0,0.0,0.215285,0.464911,0.075512,0.060410,0.204030,20230314,HOT_COLD_3_0,574013f14fdf4798208bba26,23
142150,cold,Ambar Chreuvu,Thursday,1800,Thursday-1800,0,0.0,0,0.0,0.0,0.0,0.0,0.325739,0.391485,0.252629,0.176578,0.286608,20230314,HOT_COLD_3_0,574013f14fdf4798208bba26,18
142151,medium,Ambar Chreuvu,Thursday,1600,Thursday-1600,0,0.0,0,0.0,0.0,0.0,0.0,0.407485,0.473703,0.320591,0.270024,0.367951,20230314,HOT_COLD_3_0,574013f14fdf4798208bba26,16
142152,cold,Ambar Chreuvu,Thursday,2000,Thursday-2000,0,0.0,0,0.0,0.0,0.0,0.0,0.372918,0.454045,0.227751,0.143103,0.299454,20230314,HOT_COLD_3_0,574013f14fdf4798208bba26,20


In [26]:
df_morning = df_morning.merge(df_mismatch[['weekday_name', 'hour', 'pickup_cluster', 'cluster_hotness_score']],
                               on = ['weekday_name', 'hour', 'pickup_cluster'], how = 'left')
df_evening = df_evening.merge(df_mismatch[['weekday_name', 'hour', 'pickup_cluster', 'cluster_hotness_score']],
                               on = ['weekday_name', 'hour', 'pickup_cluster'], how = 'left')
df_evening.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,customer_location_latitude,pickup_location_hex_8,pickup_location_latitude,pickup_location_longitude,drop_location_hex_8,drop_location_latitude,drop_location_longitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,second,minute,date,count,pickup_cluster,drop_cluster,num_of_orders,weekday_name,cluster_hotness_score
0,6405e723dffc1036eb142200,1.765,8860b52e89fffff,17.376173,78.553596,Hyderabad,8860a25b61fffff,17.396656,8860a25b61fffff,17.396582,78.514220,8860b52e89fffff,17.376173,78.553596,78.514290,184411,18,1830,Link,18:59,1,20230306,dropped,None,11,44,06,1,Amberpet,Nagole,33,Monday,0.215378
1,6405e72cce0f880cb87f85d8,0.338,8860a259e5fffff,17.454533,78.424805,Hyderabad,8860b196ebfffff,17.491140,8860b196ebfffff,17.491148,78.450874,8860a259e5fffff,17.454533,78.424805,78.450874,184420,18,1830,Link,18:59,1,20230306,dropped,Long Pickup,20,44,06,1,Quthbullapur,Mothinagar,19,Monday,0.311283
2,6405e7581215c13df1c82fe1,0.621,8860a24921fffff,17.558580,78.347733,Hyderabad,8860a24921fffff,17.558582,8860b196d9fffff,17.456558,78.433543,8860b196d9fffff,17.457048,78.433361,78.347740,184504,18,1845,Link,18:59,1,20230306,customerCancelled,Found better price elsewhere,04,45,06,1,Erragadda Sunday Market,Erragadda Sunday Market,29,Monday,NaN
3,6405e78f01b8b86aa8bd578d,0.374,8860a25963fffff,17.446512,78.476143,Hyderabad,8860a25903fffff,17.434916,8860a25903fffff,17.434938,78.450502,8860a25963fffff,17.446512,78.476143,78.450478,184559,18,1845,Link,18:59,1,20230306,dropped,None,59,45,06,1,Begumpet,Begumpet,95,Monday,0.779632
4,6405e7b1906657075b0e1ae3,2.733,8860b196e5fffff,17.499882,78.459274,Hyderabad,8860b196cdfffff,17.469910,8860b196cdfffff,17.469909,78.458405,8860b196e5fffff,17.499882,78.459274,78.458405,184633,18,1845,Link,18:59,1,20230306,dropped,None,33,46,06,1,Quthbullapur,Jeedimetla Industrial Estate,1,Monday,0.311283


In [27]:
df_morning_filter = df_morning[['captain_location_hex_8', 'captain_location_latitude', 'captain_location_longitude', 'pickup_location_hex_8',
 'pickup_location_latitude', 'pickup_location_longitude', 'pickup_cluster', 'hour', 'yyyymmdd', 'drop_location_hex_8', 'drop_location_latitude',
  'drop_location_longitude', 'num_of_orders', 'drop_cluster', 'weekday_name', 'cluster_hotness_score']]
df_evening_filter = df_evening[['captain_location_hex_8', 'captain_location_latitude', 'captain_location_longitude', 'pickup_location_hex_8',
 'pickup_location_latitude', 'pickup_location_longitude', 'pickup_cluster', 'hour', 'yyyymmdd', 'drop_location_hex_8', 'drop_location_latitude',
  'drop_location_longitude', 'num_of_orders', 'drop_cluster', 'weekday_name', 'cluster_hotness_score']]

In [28]:
df_evening_filter.to_csv('hyderabad_demand_supply_evening_vw2.csv', index = False)
df_morning_filter.to_csv('hyderabad_demand_supply_morning_vw2.csv', index = False)

In [17]:
#Query data for online captains

q_online_captains = """
select
        yyyymmdd, substr(hhmm, 1, 2) as hour, cluster,
        count(distinct captain_id) as Online_Captains
from 
(select 
        a.*, city, cluster
        
from 
        hive.datasets.supplycursory_history a 
        left join datasets.city_cluster_hex b 
        on a.location = b.hex_id and a.resolution = cast(b.resolution as varchar)
where 
        yyyymmdd >= '20230227'
        and yyyymmdd <= '20230302'
        and city in ('Chennai')
        and a.resolution = '8' 
        and service in ('Link')
        and substr(hhmm, 1, 2) in ('17', '18', '19', '20', '21')
        and location in ('88618c4f07fffff', '88618c4f05fffff', '88618c48d1fffff', '88618c4ab3fffff', '88618c4f29fffff', '88618c4f65fffff', '88618c48d3fffff',
 '88618c4f61fffff', '88618c4f3dfffff', '88618c4f2bfffff', '88618c4f2dfffff', '88618c4f25fffff', '88618c4f35fffff', '88618c4abbfffff', '88618c48ddfffff',
   '88618c4f63fffff', '88618c4f69fffff', '88618c4d49fffff', '88618c4d4dfffff', '88618c48d5fffff', '88618c4d41fffff', '88618c4f6bfffff', '88618c48dbfffff',
     '88618c4f67fffff', '88618c4899fffff', '88618c4ab7fffff', '88618c4f01fffff', '88618c4f47fffff', '88618c4f27fffff', '88618c4f03fffff', '88618c48c3fffff',
       '88618c4f39fffff', '88618c48d9fffff', '88618c4f6dfffff', '88618c4f45fffff', '88618c4ab1fffff', '88618c48d7fffff', '88618c4f21fffff', '88618c4a97fffff',
         '88618c4f23fffff', '88618c4f4dfffff', '88618c4f0dfffff', '88618c489bfffff', '88618c48cbfffff', '88618c4f15fffff', '88618c4a91fffff', '88618c4f09fffff',
           '88618c4a93fffff', '88618c4f0bfffff')
        and cast(status AS varchar) IN ('2')
)
group by 1,2,3
"""

#Load data into pandas table
df_online_captains_copy = pd.read_sql(q_online_captains, connection)

In [18]:
#Make a copy of the dataset and view it
df_online_captains = df_online_captains_copy.copy()
df_online_captains.head()

,yyyymmdd,hour,cluster,Online_Captains
0,20230301,17,Nungambakkam,237
1,20230228,17,Nungambakkam,231
2,20230301,21,Teynampet,175
3,20230228,20,T Nagar,322
4,20230301,18,Kotturpuram,66


In [19]:
df_online_captains.groupby(['yyyymmdd', 'hour'])['Online_Captains'].sum().reset_index()

,yyyymmdd,hour,Online_Captains
0,20230227,17,1338
1,20230227,18,1595
2,20230227,19,1795
3,20230227,20,1689
4,20230227,21,1399
5,20230228,17,1257
6,20230228,18,1519
7,20230228,19,1787
8,20230228,20,1624
9,20230228,21,1371


In [20]:
#Rename cluster column
df_online_captains = df_online_captains.rename(columns = {'cluster':'pickup_cluster'})
df_online_captains.head()

,yyyymmdd,hour,pickup_cluster,Online_Captains
0,20230301,17,Nungambakkam,237
1,20230228,17,Nungambakkam,231
2,20230301,21,Teynampet,175
3,20230228,20,T Nagar,322
4,20230301,18,Kotturpuram,66


In [21]:
#Merge online captains with demand data
df_morning_grouped = df_morning_grouped.merge(df_online_captains, on = ['yyyymmdd', 'hour', 'pickup_cluster'], how = 'left')
df_evening_grouped = df_evening_grouped.merge(df_online_captains, on = ['yyyymmdd', 'hour', 'pickup_cluster'], how = 'left')
df_morning_grouped = df_morning_grouped.rename(columns = {'order_id':'num_of_orders'})
df_evening_grouped = df_evening_grouped.rename(columns = {'order_id':'num_of_orders'})
df_morning_grouped['demand_to_supply_ratio'] = round(df_morning_grouped['num_of_orders']/df_morning_grouped['Online_Captains'], 4)
df_evening_grouped['demand_to_supply_ratio'] = round(df_evening_grouped['num_of_orders']/df_evening_grouped['Online_Captains'], 4)

df_morning_grouped = df_morning_grouped.sort_values(by = 'demand_to_supply_ratio', ascending = False)
df_evening_grouped = df_evening_grouped.sort_values(by = 'demand_to_supply_ratio', ascending = False)
df_evening_grouped.head()

,yyyymmdd,hour,pickup_cluster,num_of_orders,Online_Captains,demand_to_supply_ratio
169,20230302,18,Thousand Lights Metro,92,43,2.1395
119,20230301,18,Thousand Lights Metro,87,50,1.7400
164,20230302,18,Nungambakkam,392,235,1.6681
100,20230301,17,Alwarpet,71,44,1.6136
110,20230301,18,Alwarpet,118,76,1.5526


In [27]:
df_evening_grouped.groupby(['hour'])['demand_to_supply_ratio'].mean()

hour
17    0.918695
18    1.022725
19    0.714440
20    0.515355
21    0.482165
Name: demand_to_supply_ratio, dtype: float64

In [22]:
df_morning = df_morning.merge(df_morning_grouped, on = ['yyyymmdd', 'pickup_cluster', 'hour'], how = 'left')
df_evening = df_evening.merge(df_evening_grouped, on = ['yyyymmdd', 'pickup_cluster', 'hour'], how = 'left')
df_morning.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,customer_location_latitude,pickup_location_hex_8,pickup_location_latitude,pickup_location_longitude,drop_location_hex_8,drop_location_latitude,drop_location_longitude,customer_location_longitude,hhmmss,quarter_hour,service_obj_service_name,time_bucket,weekday,order_status,cancel_reason,second,minute,date,count,drop_cluster,num_of_orders,yyyymmdd,hour,pickup_cluster,Online_Captains,demand_to_supply_ratio


In [23]:
#Create a column for weekdays
df_morning['weekday_name'] = df_morning['weekday'].apply(lambda x: calendar.day_name[(x-1)])
df_evening['weekday_name'] = df_evening['weekday'].apply(lambda x: calendar.day_name[(x-1)])
df_morning.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,customer_location_latitude,pickup_location_hex_8,pickup_location_latitude,pickup_location_longitude,drop_location_hex_8,drop_location_latitude,drop_location_longitude,customer_location_longitude,hhmmss,quarter_hour,service_obj_service_name,time_bucket,weekday,order_status,cancel_reason,second,minute,date,count,drop_cluster,num_of_orders,yyyymmdd,hour,pickup_cluster,Online_Captains,demand_to_supply_ratio,weekday_name


In [24]:
num_of_hex_morning = df_morning.groupby(['pickup_cluster'])['pickup_location_hex_8'].nunique().reset_index()
num_of_hex_evening = df_evening.groupby(['pickup_cluster'])['pickup_location_hex_8'].nunique().reset_index()
num_of_hex_morning = num_of_hex_morning.rename(columns = {'pickup_location_hex_8':'num_of_hexes'})
num_of_hex_evening = num_of_hex_evening.rename(columns = {'pickup_location_hex_8':'num_of_hexes'})

df_morning = df_morning.merge(num_of_hex_evening, on = ['pickup_cluster'], how = 'left')
df_evening = df_evening.merge(num_of_hex_evening, on = ['pickup_cluster'], how = 'left')
df_morning['online_captains_per_hex'] = round(df_morning['Online_Captains']/df_morning['num_of_hexes'], 4)
df_evening['online_captains_per_hex'] = round(df_evening['Online_Captains']/df_evening['num_of_hexes'], 4)

In [25]:
df_morning_filter = df_morning[['captain_location_hex_8', 'captain_location_latitude', 'captain_location_longitude', 'pickup_location_hex_8',
 'pickup_location_latitude', 'pickup_location_longitude', 'pickup_cluster', 'hour', 'yyyymmdd', 'drop_location_hex_8', 'drop_location_latitude',
  'drop_location_longitude', 'num_of_orders', 'drop_cluster', 'Online_Captains', 'demand_to_supply_ratio', 'weekday_name', 'online_captains_per_hex']]
df_evening_filter = df_evening[['captain_location_hex_8', 'captain_location_latitude', 'captain_location_longitude', 'pickup_location_hex_8',
 'pickup_location_latitude', 'pickup_location_longitude', 'pickup_cluster', 'hour', 'yyyymmdd', 'drop_location_hex_8', 'drop_location_latitude',
  'drop_location_longitude', 'num_of_orders', 'drop_cluster', 'Online_Captains', 'demand_to_supply_ratio', 'weekday_name', 'online_captains_per_hex']]

In [26]:
df_evening_filter.to_csv('chennai_demand_supply_evening.csv', index = False)